### Create Training Data (TFRecords)
In order to train efficiently, the data created in Spark needs to be transformed into TFRecords. This code is run after the "training_data_create_SPARK.ipynb" notebook is run.

In [16]:
# !pip install tensorflow==2.4.1
# !pip install transformers
# !pip install pyarrow

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from math import ceil

In [2]:
def check_targets(targs):
    if targs[0] == -1:
        return 1
    else:
        return 0

In [4]:
def create_tfrecords_dataset(data, iter_num, dataset_type='train'):
    data['no_target'] = data['target_tok'].apply(check_targets)
    data = data[data['no_target']==0].copy()
    
    paper_title = tf.ragged.constant(data['paper_title_tok'].to_list())
    
    targets = tf.keras.preprocessing.sequence.pad_sequences(data['target_tok'].to_list(), maxlen=20, 
                                                            dtype='int64', padding='post', 
                                                            truncating='post', value=0)

    ds = tf.data.Dataset.zip((tf.data.Dataset.from_tensor_slices(paper_title),
                              tf.data.Dataset.from_tensor_slices(data['journal_tok'].to_list()),
                              tf.data.Dataset.from_tensor_slices(data['doc_type_tok'].to_list()),
                              tf.data.Dataset.from_tensor_slices(targets)))
    
    serialized_features_dataset = ds.map(tf_serialize_example)
    
    filename = f"./iteration_final/basic_word_tokenized/tfrecords/{dataset_type}/{str(iter_num).zfill(4)}.tfrecord"
    writer = tf.data.experimental.TFRecordWriter(filename)
    writer.write(serialized_features_dataset)

In [5]:
def tf_serialize_example(f0, f1, f2, f3):
    tf_string = tf.py_function(serialize_example, (f0, f1, f2, f3), tf.string)
    return tf.reshape(tf_string, ())

In [6]:
def serialize_example(paper_title, journal, doc_type, targets):
    paper_title_list = tf.train.Int64List(value=paper_title.numpy().tolist())
    journal_list = tf.train.Int64List(value=journal.numpy().tolist())
    doc_type_list = tf.train.Int64List(value=doc_type.numpy().tolist())
    targets_list = tf.train.Int64List(value=targets.numpy().tolist())
    
    paper_title_feature = tf.train.Feature(int64_list = paper_title_list)
    journal_feature = tf.train.Feature(int64_list = journal_list)
    doc_type_feature = tf.train.Feature(int64_list = doc_type_list)
    targets_feature = tf.train.Feature(int64_list = targets_list)
    
    features_for_example = {
        'paper_title': paper_title_feature,
        'journal': journal_feature,
        'doc_type': doc_type_feature,
        'targets': targets_feature
    }
    
    example_proto = tf.train.Example(features=tf.train.Features(feature=features_for_example))
    
    return example_proto.SerializeToString()

In [7]:
def turn_part_file_into_tfrecord(base_path, dataset_type='train'):
    file_list = [x for x in os.listdir(f"{base_path}{dataset_type}") if x.endswith('parquet')]
    file_list.sort()
    print(f"There are {len(file_list)} files for {dataset_type}")
    for i, file_name in enumerate(file_list):
        data = pd.read_parquet(f"{base_path}{dataset_type}/{file_name}")
        print(f"_____File number: {i} ({data.shape[0]} samples)")
        create_tfrecords_dataset(data, i, dataset_type)

In [8]:
base_file_path = f"./iteration_final/basic_word_tokenized/tokenized_data/"

#### Without padding

In [12]:
%%time
turn_part_file_into_tfrecord(base_file_path, 'train')

There are 50 files for train
_____File number: 0 (1955629 samples)


2021-11-05 22:19:01.825826: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


_____File number: 1 (1954544 samples)
_____File number: 2 (1954550 samples)
_____File number: 3 (1953478 samples)
_____File number: 4 (1954209 samples)
_____File number: 5 (2135999 samples)
_____File number: 6 (1955170 samples)
_____File number: 7 (1953203 samples)
_____File number: 8 (2134660 samples)
_____File number: 9 (1953918 samples)
_____File number: 10 (1954122 samples)
_____File number: 11 (1954301 samples)
_____File number: 12 (1954050 samples)
_____File number: 13 (1952498 samples)
_____File number: 14 (1951550 samples)
_____File number: 15 (1954276 samples)
_____File number: 16 (1953034 samples)
_____File number: 17 (1953935 samples)
_____File number: 18 (1954015 samples)
_____File number: 19 (2133465 samples)
_____File number: 20 (1955204 samples)
_____File number: 21 (1953749 samples)
_____File number: 22 (1954460 samples)
_____File number: 23 (1954142 samples)
_____File number: 24 (1952285 samples)
_____File number: 25 (2132684 samples)
_____File number: 26 (1951397 samp

In [13]:
%%time
turn_part_file_into_tfrecord(base_file_path, 'val')

There are 10 files for val
_____File number: 0 (61288 samples)
_____File number: 1 (62102 samples)
_____File number: 2 (30503 samples)
_____File number: 3 (31457 samples)
_____File number: 4 (24694 samples)
_____File number: 5 (29589 samples)
_____File number: 6 (60880 samples)
_____File number: 7 (30634 samples)
_____File number: 8 (57063 samples)
_____File number: 9 (59699 samples)
CPU times: user 2min 27s, sys: 9.69 s, total: 2min 37s
Wall time: 2min 10s


In [14]:
%%time
turn_part_file_into_tfrecord(base_file_path, 'test')

There are 5 files for test
_____File number: 0 (20395 samples)
_____File number: 1 (24338 samples)
_____File number: 2 (24756 samples)
_____File number: 3 (17072 samples)
_____File number: 4 (10677 samples)
CPU times: user 32.5 s, sys: 2.1 s, total: 34.6 s
Wall time: 28.7 s
